# Pollution estimation with improved accuracy using a combination of hyper-spectral satellite imagery data and maps

## Problem Statement

#### Using satellite imagery along with the map data from the open street map to find the regression estimate of pollution (Nitrogen Dioxide) in the region for specific time instances

### Approach

This hackathon Is very challenging and interesting at the same time because we not only have to make it computationally efficient but also preserve features importance. Considering so many features for regression model, it was difficult task to develop a model that deals so huge sets of features. We divided our project into two parts. First part was data processing and compression, and second part was modelling. Data were extracted from NetCDF format using Panoply. Extracted data consisted of data for whole world. We had to extract information from that, so we made a function which captures the nearest coordinates (to coordinates given by user) and return features vector of that particular location and then converting them to data frame and then finally to array which can be used by model.
 Our approach was to break model into three parts or take three sets of factors into account. Each model takes a definite set of features. These models are then merge together to get a hybrid neural network architecture.


![Image](hack.png)

In [437]:
## model 2 feature extraction
import numpy as np
import geopy.distance
import sys
import numpy as np
import time
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import smopy
import cv2
from tensorflow.keras import datasets, layers, models
from tensorflow.keras.layers import Dense, Dropout, Input,concatenate
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten
from tensorflow.keras.models import Model
import keras,os
from keras.models import Sequential
#from keras.layers import Dense, Conv2D, MaxPool2D , Flatten
from keras.preprocessing.image import ImageDataGenerator

## Important functions

#### NetCDF data contains data from whole world. coordinates of a location is accessed by spanline and groundpixel. so this function will return spanline and ground pixel of netcdf data which can be further used to get feautes of particular location

In [330]:
def get_span_grid(latitude,longitude):
    distance_between={}
    a=latitude
    b=longitude
    for i in coord_local.columns:
        for j in coord_local.index:
            coords_1=(a,b)
            try:
                coords_2=(float(coord_local[i][j].split(',')[0]),float(coord_local[i][j].split(',')[1]))
                dist_calc=geopy.distance.distance(coords_1, coords_2).km
                #print(dist_calc)
                distance_between[float(dist_calc)]=(i,j)
            except AttributeError:
                continue
    
    grid,span=distance_between[min(distance_between.keys())]
    return grid,span

#### Compressing coordinates data for london and nearby. Since having coordinates for each location of world is not feasable and make data size huge, so this function will narrow down coordinates

In [367]:

def extract(coordinates):
    latitude_1=50.0
    latitude_2=52
    longitude_1=-1
    longitude_2=2
    lat=float(coordinates.split(',')[0])
    lon=float(coordinates.split(',')[1])
    if lat > latitude_1 and lat < latitude_2 and lon > longitude_1 and lon < longitude_2:
        return str(lat)+','+str(lon)
    else:
        return 0

#### Getting features of satellite data through this function which automatically takes coordinates using get_span_grid fuunction to get spanline and ground pixel

In [ ]:
def get_hyper_Spectral_features(lat,lon):
    ground_pixel,span_line=get_span_grid(lat,lon)
    feature1=air_mass_factor_troposphere.iloc[span_line][ground_pixel]
    feature2=cloud_radiance_fraction_nitrogendioxide_window.iloc[span_line][ground_pixel]
    feature3=nitrogendioxide_tropospheric_column.iloc[span_line][ground_pixel]
    feature4=water_liquid_slant_column_density.iloc[span_line][ground_pixel]
    hyper_spectral_features=[feature1,feature2,feature3,feature4]
    return hyper_spectral_features

#### Converting input time to suitable format

In [320]:
def convert_date(date_string):
    convert_date=datetime.strptime(date_string, '%B %d %Y %H:%M%p')
    convert_date=convert_date.strftime('%d/%m/%Y %H:%M:%S')
    convert_date=datetime.strptime(convert_date, '%d/%m/%Y %H:%M:%S')
    return convert_date

#### Helper function to remove bad data which cannot be converted to float

In [492]:
def remove_bad_data(x):
    try:
        return float(x)
        
    except ValueError:
        return 0

#### To get nearest coordinates for time series data

In [323]:
def get_coord_pointer(lat,lon):
    coord_1=(lat,lon)
    array_coord = os.listdir('data1/timeseries')
    dist=10000
    for i in array_coord:
        coord_2=(float(i.split('_')[0]),i.split('_')[1].split('.csv')[0])
        distance=geopy.distance.distance(coord_1, coord_2).km
        if distance <dist:
            dist=distance
            coord_pointer=i
            
    return coord_pointer

In [473]:
#get distance also
def get_coord_pointer_dist(lat,lon):
    coord_1=(lat,lon)
    array_coord = os.listdir('data1/timeseries')
    dist=10000
    for i in array_coord:
        coord_2=(float(i.split('_')[0]),i.split('_')[1].split('.csv')[0])
        distance=geopy.distance.distance(coord_1, coord_2).km
        if distance <dist:
            dist=distance
            coord_pointer=i
            
    return coord_pointer,dist

## Extracting and compressing data


##### Note: Do not run ths because this was raw data and then later it compressed. So if you want to explore compressed data skip this section. you can also look folder features.

In [277]:
# Dataset consits of data for all world so we need to narrow down location till U.K

air_mass_factor_troposphere=pd.read_csv('data1/features/air_mass_factor_troposphere.csv')
cloud_radiance_fraction_nitrogendioxide_window=pd.read_csv('data1/features/cloud_radiance_fraction_nitrogendioxide_window.csv')
nitrogendioxide_tropospheric_column=pd.read_csv('data1/features/nitrogendioxide_tropospheric_column.csv')
water_liquid_slant_column_density=pd.read_csv('data1/features/water_liquid_slant_column_density.csv')
air_mass_factor_troposphere


,1,2,3,4,5,6,7,8,9,10,...,441,442,443,444,445,446,447,448,449,450
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3730,NaN,3.996674,4.035570,4.064984,4.067941,4.033320,3.947936,3.814722,3.649008,3.642426,...,2.635669,2.613383,2.593173,2.573021,2.544679,2.507278,2.471988,2.428825,2.375102,2.327318
3731,NaN,4.014929,3.988487,3.998895,4.013307,3.969422,3.905218,3.778103,3.642178,3.592658,...,2.608702,2.600179,2.605427,2.584003,2.547207,2.507121,2.471816,2.433708,2.385399,2.337814
3732,NaN,3.931113,3.907353,3.928293,3.967877,3.971287,3.811789,3.764996,3.806537,3.805441,...,2.612480,2.616801,2.619986,2.591538,2.554092,2.504519,2.471715,2.442029,2.394749,2.357204
3733,NaN,3.926278,3.838709,3.944358,3.986444,3.971414,3.850075,3.843834,3.880197,3.857552,...,2.623158,2.625378,2.615728,2.593893,2.557548,2.511056,2.473781,2.433137,2.378527,2.351014


### Reoving coordinates of whole world except region of interest (skip)

In [ ]:
coord=pd.read_csv('data1/geographical_data/coord.csv')

coord_local=coord.copy()
for i in coord_local.columns.tolist():
    coord_local[i]=coord_local[i].apply(lambda x: extract(x))
    
coord_local=coord_local.loc[~(coord_local==0).all(axis=1)]
coord_local=coord_local.loc[~(coord_local==0).all(axis=1)]

list_drop=[]

for i in coord_local.columns:
    #coord_local[i]=coord_local[i].apply(lambda x:float(x))
    try:
        coord_local[i]=coord_local[i].apply(lambda x:float(x))
        
    except ValueError:
        continue
    try:
        if coord_local[i].sum()==0:
            list_drop.append(i)
    except TypeError:
        continue
coord_local=coord_local.drop(list_drop, axis=1)


### compressing data (skip)

In [278]:
list_range=list(range(75,450))
list_final=[]
for i in list_range:
    list_final.append(str(i))
    
list_range=list(range(3110,3734))
row_final=[]
for i in list_range:
    row_final.append(i)
    
air_mass_factor_troposphere[0:2057]=0
air_mass_factor_troposphere[3110:]=0
air_mass_factor_troposphere[list_final]=0
air_mass_factor_troposphere=air_mass_factor_troposphere.drop(columns=list_final)
air_mass_factor_troposphere=air_mass_factor_troposphere.drop(row_final)




cloud_radiance_fraction_nitrogendioxide_window[0:2057]=0
cloud_radiance_fraction_nitrogendioxide_window[3110:]=0
cloud_radiance_fraction_nitrogendioxide_window[list_final]=0
cloud_radiance_fraction_nitrogendioxide_window=cloud_radiance_fraction_nitrogendioxide_window.drop(columns=list_final)
cloud_radiance_fraction_nitrogendioxide_window=cloud_radiance_fraction_nitrogendioxide_window.drop(row_final)


nitrogendioxide_tropospheric_column[0:2057]=0
nitrogendioxide_tropospheric_column[3110:]=0
nitrogendioxide_tropospheric_column[list_final]=0
nitrogendioxide_tropospheric_column=nitrogendioxide_tropospheric_column.drop(columns=list_final)
nitrogendioxide_tropospheric_column=nitrogendioxide_tropospheric_column.drop(row_final)




water_liquid_slant_column_density[0:2057]=0
water_liquid_slant_column_density[3110:]=0
water_liquid_slant_column_density[list_final]=0
water_liquid_slant_column_density=water_liquid_slant_column_density.drop(columns=list_final)
water_liquid_slant_column_density=water_liquid_slant_column_density.drop(row_final)

### Saving compressed data (skip)

In [279]:
air_mass_factor_troposphere.to_excel('air_mass_factor_troposphere.xlsx')
cloud_radiance_fraction_nitrogendioxide_window.to_excel('cloud_radiance_fraction_nitrogendioxide_window.xlsx')
nitrogendioxide_tropospheric_column.to_excel('nitrogendioxide_tropospheric_column.xlsx')
water_liquid_slant_column_density.to_excel('water_liquid_slant_column_density.xlsx')

### Reading  compressed data 

In [284]:
# Reading features from compressed file
air_mass_factor_troposphere=pd.read_excel('data1/features/air_mass_factor_troposphere.xlsx')
cloud_radiance_fraction_nitrogendioxide_window=pd.read_excel('data1/features/cloud_radiance_fraction_nitrogendioxide_window.xlsx')
nitrogendioxide_tropospheric_column=pd.read_excel('data1/features/nitrogendioxide_tropospheric_column.xlsx')
water_liquid_slant_column_density=pd.read_excel('data1/features/water_liquid_slant_column_density.xlsx')

In [243]:
# Readinf coordinates(compressed) data
coord_local.to_excel('extracted_features/coordinates.xlsx')
c=pd.read_excel('data1/geographical_data/coordinates.xlsx')
coord_local=c.set_index('Unnamed: 0')

## size of extracted features

Extracted features are stored in 'data1' folder

In [312]:
Folderpath = 'data1/features'  
 
size = 0
for i in os.scandir(Folderpath):
    size+=os.path.getsize(i)
Folderpath = 'data1/geographical_data'  
for i in os.scandir(Folderpath):
    size+=os.path.getsize(i)
Folderpath = 'data1/timeseries' 
for i in os.scandir(Folderpath):
    size+=os.path.getsize(i)
size=size/1000
print(str(size)+'kb')

4703.642kb


# Training dataset
#### Trained model already saved in my_model folder 

#### Processing map data 

In [19]:
# Checking sample from map and doing analysis
lat=51.20
lon=-0.25
LatMin=lat-0.1
LatMax=lat+0.1
LngMin=lon-0.1
LngMax=lon+0.1
map = smopy.Map((LatMin,LngMin, LatMax,LngMax))

## Deciding na,e of photo which will be saved for further processing
name=str(lat)+'_'+str(lon)+'.png'
map.save_png(name)
img = cv2.imread(name)

resized_image = cv2.resize(img, (60, 60)) 
resized_image.shape

Lowered zoom level to keep map size reasonable. (z = 11)


# Model structure

## CNN model for map data

In [315]:
#model1 using tensorflow functional API
input1=Input(shape=(60, 60, 3)) # input shape
y1=Conv2D(32, (3, 3), activation='relu', input_shape=(60, 60, 3))(input1)
y2= MaxPooling2D()(y1)
y3=Conv2D(64, (3, 3), activation='relu')(y2)
y4= Flatten()(y3)
output1=Dense(10, activation="relu")(y4)

## Model for last 7 days data and data from satellite

In [316]:
#model2 Last 7 days input (24x7) hourly
input2=Input(shape=(168,))
z1=Dense(5, activation="relu")(input2)

#model3 input features from satellite
input3=Input(shape=(4,))
x=Dense(1, activation="relu")(input3)

## Merging all models on second last Dense layer

In [317]:
merge = concatenate([output1, z1,x])
out=Dense(1, activation="relu")(merge)

### Defining model

In [318]:
model=Model(inputs=[input1,input2,input3],outputs=[out])
model.compile(optimizer='adam', loss='mean_absolute_error')

### Getting time series data for trainng

In [275]:
#Time series
London_Bexley=(51.462900, 0.139400)
df=pd.read_csv('data1/timeseries/51.462900_ 0.139400.csv')

#### Processing data

In [276]:
df=df.dropna()
df['date']=df['Date']+' '+df['Time']
from datetime import datetime
df['date'] = df['date'].apply(lambda x: datetime.strptime(x, '%d/%m/%Y %H:%M:%S'))

In [279]:
# checking function
#given_date=%B %d %Y %H:%M%p
given_date='March 25 2021 11:00AM'
convert_date=datetime.strptime(given_date, '%B %d %Y %H:%M%p')
convert_date=convert_date.strftime('%d/%m/%Y %H:%M:%S')

In [271]:
convert_date

'25/03/2021 11:00:00'

In [289]:
#given coordinates and date and time, extract time series and dependent variable
import os
arr = os.listdir('data1/timeseries')

In [306]:
import geopy.distance
lat_1=51.522918

lon_1=-0.130821
coord_1=(lat_1,lon_1)
#distgeopy.distance.distance(coords_1, coords_2).km

dist=10000
for i in arr:
    coord_2=(float(i.split('_')[0]),i.split('_')[1].split('.csv')[0])
    #print(coord_2)
    distance=geopy.distance.distance(coord_1, coord_2).km
    #print(distance)
    if distance <dist:
        dist=distance
        coord_pointer=i


In [307]:
coord_pointer

'51.522918_-0.130821.csv'

In [313]:
df =pd.read_csv('data1/timeseries/'+coord_pointer)
df=df.dropna()
df['date']=df['Date']+' '+df['Time']
from datetime import datetime
df['date'] = df['date'].apply(lambda x: datetime.strptime(x, '%d/%m/%Y %H:%M:%S'))

In [347]:
given_time= 'March 26 2021 11:00AM'
time=convert_date(given_time)

In [330]:
y= float(df[df.date == time]['Nitrogen dioxide'])

In [350]:
hours = 168
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta
hours_added = timedelta(hours = -hours)
time_past=time + hours_added
#future_date_and_time = current_date_and_time + hours_added
time_past

datetime.datetime(2021, 3, 19, 11, 0)

In [385]:
x= df[df['date'] < time]#['Nitrogen dioxide']
x=x[x['date'] >= time_past]['Nitrogen dioxide']
#x=x.apply(lambda y:float(y))
x=list(x)
X=[]
for i in x:
    try:
        X.append(float(i))
    except ValueError:
        X.append(statistics.mean(X))

In [121]:
import pandas as pd
df =pd.read_csv('data1/timeseries/'+'51.522918_-0.130821.csv')
df=df.dropna()
df['date']=df['Date']+' '+df['Time']
from datetime import datetime
df['date'] = df['date'].apply(lambda x: datetime.strptime(x, '%d/%m/%Y %H:%M:%S'))

In [122]:
time_ran=['AM','PM']
month_ran=['March','April']
day_ran=list(range(8,30))
time1_ran=list(range(1,12))


In [438]:
import random
import statistics
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta
given_time= random.choice(month_ran)+ ' '  + str(random.choice(day_ran))+ ' ' + '2021' + ' ' + str(random.choice(time1_ran)) +':'+'00'+random.choice(time_ran)

In [398]:
import pandas as pd
df =pd.read_csv('data1/timeseries/'+'51.522918_-0.130821.csv')
df=df.dropna()
df['date']=df['Date']+' '+df['Time']
from datetime import datetime
df['date'] = df['date'].apply(lambda x: datetime.strptime(x, '%d/%m/%Y %H:%M:%S'))
import smopy
import cv2

lat=51.522918
lon=-0.130821
LatMin=lat-0.1
LatMax=lat+0.1
LngMin=lon-0.1
LngMax=lon+0.1
map = smopy.Map((LatMin,LngMin, LatMax,LngMax))
name=str(lat)+'_'+str(lon)+'.png'
map.save_png(name)
img = cv2.imread(name)
resized_image = cv2.resize(img, (60, 60)) 
resized_image.shape
spec_features=get_hyper_Spectral_features(lat,lon)

Lowered zoom level to keep map size reasonable. (z = 12)


### Training dataset preparation (skip)

In [428]:
x_array=[]
y_array=[]
x_hyper=[]
x_map=[]
for i in range(15000):
    x_map.append(resized_image)
    x_hyper.append(spec_features)
    temp=0
    given_time= random.choice(month_ran)+ ' '  + str(random.choice(day_ran))+ ' ' + '2021' + ' ' + str(random.choice(time1_ran)) +':'+'00'+random.choice(time_ran)
    time=convert_date(given_time)
    try:
        y= float(df[df.date == time]['Nitrogen dioxide'])
    except ValueError:
        y =0
    
    hours = 168
    hours_added = timedelta(hours = -hours)
    time_past=time + hours_added
    #future_date_and_time = current_date_and_time + hours_added
    time_past
    x= df[df['date'] < time]#['Nitrogen dioxide']
    x=x[x['date'] >= time_past]['Nitrogen dioxide']
    x=list(x)
    X=[]
    for i in x:
        try:
            X.append(float(i))
        except ValueError:
            X.append(10) #very few cases where due to some error string is not converted to float
            temp=X.copy()
    y_array.append(y)
    x_array.append(X)

In [371]:
#x_array

# Model Training (skip as trained model available)

In [427]:
#model1
from tensorflow.keras.layers import Dense, Dropout, Input,concatenate
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten
#from keras.optimizers import SGD
from tensorflow.keras.models import Model
input1=Input(shape=(60, 60, 3)) # input layer
y1=Conv2D(32, (3, 3), activation='relu', input_shape=(60, 60, 3))(input1)  #convolution layer
y2= MaxPooling2D()(y1)  #maxpooling layer
y3=Conv2D(64, (3, 3), activation='relu')(y2)
y4= Flatten()(y3)
output1=Dense(10, activation="relu")(y4)

#model2
input2=Input(shape=(5,))
z1=Dense(168, activation="relu")(input2)
#model3
input3=Input(shape=(4,))
x=Dense(1, activation="relu")(input3)

merge = concatenate([output1, z1,x])
out=Dense(1, activation="relu")(merge)

model=Model(inputs=[input1,input2,input3],outputs=[out])
opt = tf.keras.optimizers.Adam(learning_rate=0.001)
model.compile(optimizer=opt, loss='mean_absolute_error')



## To use eagerly execution

In [408]:
tf.config.run_functions_eagerly(True)
tf.data.experimental.enable_debug_mode()

In [423]:
x_array_np=np.array(x_array)
y_array_np=np.array(y_array)
x_hyper_np=np.array(x_hyper)
x_map_np=np.array(x_map)

#### Model training

In [508]:
model.fit([x_map_np,x_array_np,x_hyper_np],y_array_np)

# Model prediction

## Importanct functions

In [328]:
model.save('my_model') 

INFO:tensorflow:Assets written to: my_model\assets


### Load model weight (Run this section)

In [336]:
from tensorflow.keras.layers import Dense, Dropout, Input,concatenate
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten
#from keras.optimizers import SGD
from tensorflow.keras.models import Model
input1=Input(shape=(60, 60, 3))
y1=Conv2D(32, (3, 3), activation='relu', input_shape=(60, 60, 3))(input1)
y2= MaxPooling2D()(y1)
y3=Conv2D(64, (3, 3), activation='relu')(y2)
y4= Flatten()(y3)
output1=Dense(10, activation="relu")(y4)

#model2
input2=Input(shape=(5,))
z1=Dense(5, activation="relu")(input2)
#model3
input3=Input(shape=(4,))
x=Dense(1, activation="relu")(input3)

merge = concatenate([output1, z1,x])
out=Dense(1, activation="relu")(merge)

#model_load=Model(inputs=[input1,input2,input3],outputs=[out])
#opt = tf.keras.optimizers.Adam(learning_rate=0.001)
#model.compile(optimizer=opt, loss='mean_absolute_error')

In [429]:
new_model = tf.keras.models.load_model('my_model')

# Check its architecture
new_model.summary()

Model: "model_5"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_27 (InputLayer)           [(None, 60, 60, 3)]  0                                            
__________________________________________________________________________________________________
conv2d_25 (Conv2D)              (None, 58, 58, 32)   896         input_27[0][0]                   
__________________________________________________________________________________________________
max_pooling2d_15 (MaxPooling2D) (None, 29, 29, 32)   0           conv2d_25[0][0]                  
__________________________________________________________________________________________________
conv2d_26 (Conv2D)              (None, 27, 27, 64)   18496       max_pooling2d_15[0][0]           
____________________________________________________________________________________________

# Prediction

### Input formats in model

- Input latitude in float format
- Input longitude in float format
- Input time in following format '21 March 2021 12:00AM'



### prediction=predict_NO2(latitude,longitude,time)
## Example
`prediction=predict_NO2(51.472830, -0.013733,'March 21 2021 12:00AM')`

#### Note: Enter date between 9th March 2021 to 30th April 2022 (due to limitation of data given in challenge). It takes last 7 days data as time series input

In [503]:

        
        

def predict_NO2(input_latitude,input_longitude,input_time):
    hyper_spec_features=get_hyper_Spectral_features(input_latitude,input_longitude)
    hyper_spec_features=np.array(hyper_spec_features)
    hyper_spec_features=hyper_spec_features[ np.newaxis, :]

    df =pd.read_csv('data1/timeseries/'+ get_coord_pointer_dist(input_latitude,input_longitude)[0])
    df=df.dropna()
    
    #df.to_excel('extracted_data/timeseries.xlsx')
   
    df['date']=df['Date']+' '+df['Time']
    
    from datetime import datetime
    df['date'] = df['date'].apply(lambda x: datetime.strptime(x, '%d/%m/%Y %H:%M:%S'))
    df['Nitrogen dioxide']=df['Nitrogen dioxide'].apply(lambda x:remove_bad_data(x))
    df['Nitrogen dioxide']=get_coord_pointer_dist(51.103522,-0.604248)[1]*0.1+df['Nitrogen dioxide']

    time=convert_date(input_time)

    hours =168
    hours_added = timedelta(hours = -hours)
    hours_time_stamp = timedelta(hours = -1)
    hours_time_stamp=time+hours_time_stamp
    time_past=time + hours_added
    time_past
    x= df[df['date'] < time]#['Nitrogen dioxide']
    x=x[x['date'] >= time_past]['Nitrogen dioxide']
    x=list(x)
    last_timestamp= df[df['date'] == hours_time_stamp]['Nitrogen dioxide']
    X=[]
    for i in x:
        try:
            X.append(float(i))
        except ValueError:
             X.append(10)
    time_features=X
    time_features=np.array(time_features)
    time_features=time_features[ np.newaxis, :]

    LatMin=input_latitude-0.1
    LatMax=input_latitude+0.1
    LngMin=input_longitude-0.1
    LngMax=input_longitude+0.1
    map = smopy.Map((LatMin,LngMin, LatMax,LngMax))
    name=str(input_latitude)+'_'+str(input_longitude)+'.png'
    map.save_png(name)
    img = cv2.imread(name)
    resized_image = cv2.resize(img, (60, 60)) 
    resized_image.shape

    map_features=np.array(resized_image)
    #adding axis for single batch forecasting
    map_features=map_features[ np.newaxis, :]
    # In case you need to add a eries of input for prediction then comment out new axis becasue tensorflow forecast
    #in batches, a single sset of features will give uneven dimenensions. Thats why to bring them in one batch a new single axis has to be added
    prediction=new_model.predict([map_features,time_features,hyper_spec_features])
    final_prediction= 0.683*float(last_timestamp)+0.317*float(prediction) #Impact of last hour has 68.3% of avergae weight

    result=final_prediction
    return result

In [506]:
#import datetime
start_time = time.time()
#dt_started = datetime.datetime.utcnow()
#51.522918_-0.130821
prediction=predict_NO2(51.55650893020211, -0.35156826179036915,'March 20 2021 01:00PM')
print("--- %s seconds ---" % (time.time() - start_time))

#dt_ended = datetime.datetime.utcnow()
#print((dt_ended - dt_started).total_seconds())

Lowered zoom level to keep map size reasonable. (z = 11)
--- 1.6261248588562012 seconds ---


### Load model from saved folder

In [507]:
prediction

8.086740736639763

## Thankyou